# Baseline

基于协同过滤推荐

In [1]:
import os
import sys
def get_project_path():
    import os
    parts = os.getcwd().rsplit("machine-learning", 1)
    return os.path.join(parts[0], "machine-learning")
sys.path.append(get_project_path())
sys.path.append(os.getcwd())

from internal.pyspark.pyspark_tools import SparkSessionBuilder
spark = SparkSessionBuilder().with_minio().build()
print('Spark Version: ', spark.version)

/home/jovyan/jars/hadoop-aliyun-3.4.1.jar,/home/jovyan/jars/hadoop-aws-3.4.1.jar,/home/jovyan/jars/bundle-2.32.24.jar,/home/jovyan/jars/aliyun-sdk-oss-3.18.2.jar,/home/jovyan/jars/paimon-oss-1.2.0.jar,/home/jovyan/jars/commons-configuration2-2.12.0.jar,/home/jovyan/jars/jdom2-2.0.6.1.jar
Spark Version:  4.0.0


In [2]:
from data import DataManager
dm = DataManager('~/dataset/tianchi-news-rec')

In [3]:
dm.df_article_emb.describe()

,article_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249
count,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,...,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000,364047.000000
mean,182023.000000,-0.238645,-0.963334,0.118548,-0.279295,-0.068579,0.045945,-0.168319,-0.216564,-0.006810,...,-0.133287,-0.081912,-0.060347,0.023003,0.076947,0.084603,0.062819,0.099768,0.155917,-0.041094
std,105091.461061,0.451497,0.022651,0.495050,0.505067,0.506723,0.483288,0.514183,0.456211,0.519036,...,0.490597,0.449957,0.489498,0.507181,0.426143,0.467998,0.462678,0.499648,0.454356,0.458819
min,0.000000,-0.991183,-0.996455,-0.968431,-0.994966,-0.994489,-0.985974,-0.995090,-0.995926,-0.991316,...,-0.990412,-0.989408,-0.990432,-0.993626,-0.989042,-0.996902,-0.992921,-0.984733,-0.976071,-0.988213
25%,91011.500000,-0.620072,-0.974056,-0.289953,-0.718816,-0.503425,-0.354579,-0.610391,-0.604471,-0.446108,...,-0.547684,-0.445079,-0.479989,-0.404508,-0.248653,-0.267072,-0.306548,-0.313598,-0.201402,-0.420694
50%,182023.000000,-0.302581,-0.967605,0.124339,-0.391535,-0.093734,0.062636,-0.259313,-0.257692,-0.025777,...,-0.175781,-0.094113,-0.078034,0.000726,0.105649,0.133525,0.083315,0.128757,0.188355,-0.015232
75%,273034.500000,0.098015,-0.959061,0.545112,0.108320,0.345024,0.446979,0.244957,0.138783,0.432606,...,0.250641,0.270006,0.341105,0.459386,0.417347,0.461466,0.441831,0.531453,0.538111,0.334226
max,364046.000000,0.983694,-0.514728,0.998341,0.978092,0.996798,0.996343,0.990139,0.963327,0.992945,...,0.996401,0.981789,0.991332,0.995299,0.978823,0.989324,0.991445,0.997583,0.990507,0.968461


In [4]:
emb = dm.df_article_emb.iloc[:,1:]
metric = emb.values.astype("float32")
print(metric.shape)

(364047, 250)


In [5]:
print(metric.shape[0])

364047


# ItemCF

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

# 分批计算
sim_metrics = []
batch_size = 2000
article_count = metric.shape[0]
loop_count = int(article_count / batch_size + 1)
for i in range(loop_count):
    start = batch_size * i
    end = min(article_count, batch_size * (i+1))
    sim_metrics.append(cosine_similarity(emb[start:end], emb))

print("相似度矩阵：")
print(sim_metrics[0])

相似度矩阵：
[[ 1.          0.20460139  0.21395829 ... -0.13713181 -0.10750157
  -0.03401163]
 [ 0.20460139  1.          0.5487707  ...  0.31751916 -0.01978651
   0.0667556 ]
 [ 0.21395829  0.5487707   1.         ...  0.10459428  0.04867898
  -0.05251635]
 ...
 [ 0.16110123  0.34601498  0.61945427 ... -0.07711356  0.0828644
  -0.00110267]
 [ 0.09300059  0.1687392   0.60417902 ...  0.1892239   0.16458153
   0.14292493]
 [ 0.11926547  0.04322364  0.40970948 ...  0.10414405  0.29688822
  -0.02371853]]


In [ ]:
def recommend_by_itemcf(user_items, top_k=5):
    """
    1. 遍历用户点击的 item
    2. 对每个 item 计算 top k 个 item, 并进行拼接
    3. 最终对所有 item 按相似度排序, 并截取 top k 个
    """
    